In [3]:
import gurobipy as gb
import numpy as np
import csv

In [4]:
NUMDISTRICTS = 9

# basic voter data as numpy arrays - POPULATION, DEMOCRATS, REPUBLICANS, INDEPENDENTS
# each town is assigned an index in the idx dict; e.g. to get index of BEDFORD use idx['BEDFORD']
# to get population of BEDFORD use POP[idx['BEDFORD']]
with open("Voter_Data_MA_CSV_Version_2.csv", 'r') as votefile:
    votereader = csv.reader(votefile, delimiter=',')
    next(votereader)
    TOWNS = []
    POP = []
    DEMS = []
    REPS = []
    INDS = []
    idx = {}
    i = 0
    for row in votereader:
        town = row[0]
        TOWNS.append(town)
        idx[town] = i
        i += 1
        POP.append(int(row[-1]))
        DEMS.append(int(row[-3]))
        REPS.append(int(row[-2]))
        INDS.append(POP[-1] - DEMS[-1] - REPS[-1])
    POP = np.array(POP)
    DEMS = np.array(DEMS)
    REPS = np.array(REPS)
    INDS = np.array(INDS)

# pairwise distance data, as a numpy array
# use idx dict to get indices
with open("dist_pairs.csv", 'r') as distfile:
    # pairwise dist
    distreader = csv.reader(distfile, delimiter=',')
    DIST = np.zeros((i,i))
    for row in distreader:
        DIST[idx[row[0]]][idx[row[1]]] = float(row[-1])

# pairwise adjacency matrix (1/0 for adjacent/not adj)
# use idx dict to get indices
with open ('Adjacencies.csv', 'r') as adjfile:
    adjreader = csv.reader(adjfile, delimiter=',')
    ADJ = np.zeros((i,i))
    for row in adjreader:
        i1 = idx[row[0]]
        for t2 in row[1:]:
            if not t2 == '':
                ADJ[i1][idx[t2]] = 1

In [ ]:
# init model
m = gb.Model()

# min and max population for each district
MINPOP = 720000
MAXPOP = 735000

# big-M (for population of state)
M = 10**6

# total number of voters
VOTERS=sum(REPS)+sum(DEMS)

# init variables
# Dwin_i - whether D wins district i (0/1)
# Rwin_i - whether R wins district i (0/1)
# Ddiff_i - D wasted votes minus R wasted votes in district i (free variable)
# y - variable for removing abs. value from objective (pos float)
# z_t^i - if town t in district i (0/1)
# r_i - max dist between towns in district i, 0<=i<=9 (pos float)
Dwin = []
for i in range(NUMDISTRICTS):
    Dwin.append(m.addVar(name='Dwin'+str(i), vtype=gb.GRB.BINARY))
    
Rwin = []
for i in range(NUMDISTRICTS):
    Rwin.append(m.addVar(name='Rwin'+str(i), vtype=gb.GRB.BINARY))
    
Ddiff = []
for i in range(NUMDISTRICTS):
    Ddiff.append(m.addVar(name='Ddiff'+str(i), lb=-1e31))
    
y = m.addVar(name='y')

z = [[] for t in range(len(idx))]
for t, zt in enumerate(z):
    for i in range(NUMDISTRICTS):
        zt.append(m.addVar(name='z_'+str(t)+'('+str(i)+')', vtype=gb.GRB.BINARY))    
        
r = []
for i in range(NUMDISTRICTS):
    r.append(m.addVar(name='r'+str(i)))

m.update()

# init constraints
# each town is in exactly one district
# sum_i z_t^i = 1
for t in range(len(TOWNS)):
    m.addConstr(sum(z[t]), sense=gb.GRB.EQUAL, rhs=1., name='towndist'+str(t))
    
# district population constraint:
# MINPOP <= sum_t z_t^i * POP_i <= MAXPOP for all districts i
for i in range(NUMDISTRICTS):
    m.addConstr(sum([z[t][i]*POP[t] for t in range(len(TOWNS))]), sense=gb.GRB.LESS_EQUAL, rhs=MAXPOP, name='maxpop'+str(i))
    m.addConstr(sum([z[t][i]*POP[t] for t in range(len(TOWNS))]), sense=gb.GRB.GREATER_EQUAL, rhs=MINPOP, name='minpop'+str(i))    
    
# constraints to define Dwin and Rwin
for i in range(NUMDISTRICTS):
    m.addConstr(Dwin[i]+Rwin[i], sense=gb.GRB.EQUAL, rhs=1., name='onewins'+str(i))
    m.addConstr(sum([z[t][i]*(DEMS[t]-REPS[t]) for t in range(len(TOWNS))])-M*Dwin[i], sense=gb.GRB.LESS_EQUAL, rhs=0., name='checkDwins'+str(i))
    m.addConstr(sum([z[t][i]*(REPS[t]-DEMS[t]) for t in range(len(TOWNS))])-M*Rwin[i], sense=gb.GRB.LESS_EQUAL, rhs=0., name='checkRwins'+str(i))

# constraints to find number of wasted votes
for i in range(NUMDISTRICTS):
    m.addConstr(sum([z[t][i]*(DEMS[t]-3*REPS[t]) for t in range(len(TOWNS))])/2+2*M*Rwin[i]-Ddiff[i], sense=gb.GRB.GREATER_EQUAL, rhs=0., name='wastedgap1'+str(i))
    m.addConstr(sum([z[t][i]*(DEMS[t]-3*REPS[t]) for t in range(len(TOWNS))])/2-2*M*Rwin[i]-Ddiff[i], sense=gb.GRB.LESS_EQUAL, rhs=0., name='wastedgap2'+str(i))
    m.addConstr(sum([z[t][i]*(3*DEMS[t]-REPS[t]) for t in range(len(TOWNS))])/2+2*M*Dwin[i]-Ddiff[i], sense=gb.GRB.GREATER_EQUAL, rhs=0., name='wastedgap3'+str(i))
    m.addConstr(sum([z[t][i]*(3*DEMS[t]-REPS[t]) for t in range(len(TOWNS))])/2-2*M*Dwin[i]-Ddiff[i], sense=gb.GRB.LESS_EQUAL, rhs=0., name='wastedgap4'+str(i))

# constraints to find z (efficiency gap in # of votes)
m.addConstr(sum(Ddiff)-y, sense=gb.GRB.LESS_EQUAL, rhs=0., name='Egap1')
m.addConstr(-sum(Ddiff)-y, sense=gb.GRB.LESS_EQUAL, rhs=0., name='Egap2')

m.update()

# diameter r_i of a district is greater than every pairwise distance between towns in the district
# r_i >= dist_st * (z_s^i + z_t^i - 2) for all districts i, towns s t
for i in range(NUMDISTRICTS):
    for s in range(len(TOWNS)):
        for t in range(len(TOWNS)):
            m.addConstr(r[i] - DIST[s][t] * (z[s][i] + z[t][i] - 1), sense=gb.GRB.GREATER_EQUAL, rhs=0., name="r"+str(i)+'_'+str(s)+'-'+str(t))

# objective
print('INITIALIZING OBJECTIVE')
# min y/sum_i(r_i+d_i) (towns i)
m.update()
m.setObjective(y/VOTERS+0.0001*sum(r), gb.GRB.MINIMIZE)
print('SOLVING')
m.optimize()


INITIALIZING OBJECTIVE
SOLVING
Optimize a model with 1109243 rows, 3196 columns and 3336032 nonzeros
Variable types: 19 continuous, 3177 integer (3177 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+06]
  Objective range  [2e-07, 1e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+05]
Presolve removed 559143 rows and 9 columns (presolve time = 5s) ...
Presolve removed 559143 rows and 9 columns (presolve time = 11s) ...
Presolve removed 559143 rows and 9 columns (presolve time = 17s) ...
Presolve removed 559143 rows and 9 columns
Presolve time: 16.78s
Presolved: 550100 rows, 3187 columns, 1677530 nonzeros
Variable types: 19 continuous, 3168 integer (3168 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   1.220679e+07     26s
    7372    5.4487659e+00   0.000000e+00   2.592385e+09     30s
   14914    8.1753767e+00   0.000000e+00   1.408032e+09     36s
   20196    2.74577

KeyboardInterrupt: 

     0     2    0.00000    0  706    0.21813    0.00000   100%     -  631s


Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\jhnyb\Anaconda3\lib\site-packages\ipykernel\iostream.py", line 376, in write
    self.pub_thread.schedule(lambda : self._buffer.write(string))
  File "C:\Users\jhnyb\Anaconda3\lib\site-packages\ipykernel\iostream.py", line 203, in schedule
    self._event_pipe.send(b'')
  File "C:\Users\jhnyb\Anaconda3\lib\site-packages\zmq\sugar\socket.py", line 391, in send
    return super(Socket, self).send(data, flags=flags, copy=copy, track=track)
  File "zmq/backend/cython/socket.pyx", line 727, in zmq.backend.cython.socket.Socket.send
  File "zmq/backend/cython/socket.pyx", line 774, in zmq.backend.cython.socket.Socket.send
  File "zmq/backend/cython/socket.pyx", line 244, in zmq.backend.cython.socket._send_copy
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
    PyErr_CheckSignals()
KeyboardInterrupt


     1     4    0.00000    1  707    0.21813    0.00000   100%  8593  810s
     3     4    0.00000    2  712    0.21813    0.00000   100% 11470 4293s
     5     4 infeasible    2         0.21813    0.00000   100%  6970 4471s
     7     2 infeasible    3         0.21813    0.00000   100%  6673 4487s


In [16]:
# Code for looking at some results

print([ri.x for ri in r]) # Print values of r
print(Ddiff) # Print wasted vote differences in each district
print(y)
demarray=np.zeros(9)
reparray=np.zeros(9)
for i in range(NUMDISTRICTS):
    for t in range(len(TOWNS)):
        demarray[i]+=z[t][i].x*DEMS[t]
for i in range(NUMDISTRICTS):
    for t in range(len(TOWNS)):
        reparray[i]+=z[t][i].x*REPS[t]
print(demarray) # Print number of Democrats in each district
print(reparray) # Print number of Republicans in each district
print(sum(REPS)+sum(DEMS))

district1=np.zeros(351)
for i in range(351):
    district1[i]=z[i][0].x
print(district1) # Print vector indicating which towns are in district 1
for i in range(351):
    if district1[i]==1:
        print(TOWNS[i]) # Print list of towns in district 1

[289.0, 277.0, 285.0, 287.0, 294.0, 282.0, 272.0, 253.0, 251.0]
[<gurobi.Var Ddiff0 (value 1344.499999921958)>, <gurobi.Var Ddiff1 (value -21622.50000007957)>, <gurobi.Var Ddiff2 (value -5177.5000000741275)>, <gurobi.Var Ddiff3 (value -55764.50000007167)>, <gurobi.Var Ddiff4 (value -112820.5000000729)>, <gurobi.Var Ddiff5 (value -1909.0000000652215)>, <gurobi.Var Ddiff6 (value -9007.000000071715)>, <gurobi.Var Ddiff7 (value 121203.99999992081)>, <gurobi.Var Ddiff8 (value 17280.499999921896)>]
<gurobi.Var y (value 66471.99999526143)>
[370309. 375159. 378892. 353426. 338233. 365749. 387328. 422465. 360508.]
[122540. 139468. 129749. 154985. 187958. 123189. 135114.  60019. 108649.]
4513740
[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1.
  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0.  1. -0. -0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1.
 -0. -0. -0. 